In [113]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

adult_df = pd.read_csv("Data/adult.csv")
county_df = pd.read_csv("Data/acs2017_county_data.csv")

In [114]:
print("\nADULT_CENSUS_DATA:\n", adult_df.columns)
print("\nUS_CENSUS_COLUMNS:\n", county_df.columns)


ADULT_CENSUS_DATA:
 Index(['age', 'workclass', 'fnlwgt', 'education', 'education.num',
       'marital.status', 'occupation', 'relationship', 'race', 'sex',
       'capital.gain', 'capital.loss', 'hours.per.week', 'native.country',
       'income'],
      dtype='object')

US_CENSUS_COLUMNS:
 Index(['CountyId', 'State', 'County', 'TotalPop', 'Men', 'Women', 'Hispanic',
       'White', 'Black', 'Native', 'Asian', 'Pacific', 'VotingAgeCitizen',
       'Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment'],
      dtype='object')


### Preprocessing

In [115]:
adult_df.workclass = adult_df.workclass.apply(lambda x: 'unknown' if (x == '?') else x)
adult_df.occupation = adult_df.occupation.apply(lambda x: 'unknown' if (x == '?') else x)

# Imputation of nulls using Simple Imputer
imputer = SimpleImputer(strategy='most_frequent')
county_df['ChildPoverty'] = imputer.fit_transform(county_df[['ChildPoverty']])

# Convert Income to binary in county_df
county_df.Income = county_df.Income.apply(lambda x: '<=50K' if (x <= 50000) else '>50K')

In [116]:
# Encode categorical variables in the adult_df
le = LabelEncoder()
adult_df['workclass'] = le.fit_transform(adult_df['workclass'])
adult_df['education'] = le.fit_transform(adult_df['education'])
adult_df['marital.status'] = le.fit_transform(adult_df['marital.status'])
adult_df['occupation'] = le.fit_transform(adult_df['occupation'])
adult_df['relationship'] = le.fit_transform(adult_df['relationship'])
adult_df['race'] = le.fit_transform(adult_df['race'])
adult_df['sex'] = le.fit_transform(adult_df['sex'])
adult_df['native.country'] = le.fit_transform(adult_df['native.country'])
adult_df['income'] = le.fit_transform(adult_df['income'])

# Encode categorical variables in the county_df
le_us_census = LabelEncoder()
county_df['State'] = le_us_census.fit_transform(county_df['State'])
county_df['County'] = le_us_census.fit_transform(county_df['County'])
county_df['Income'] = le.fit_transform(county_df['Income'])

In [117]:
from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import StandardScaler

# Train logistic regression model for adult_df
X_adult = pd.DataFrame(adult_df)
y_adult = X_adult.pop('income')
#X_train_adult, X_test_adult, y_train_adult, y_test_adult = train_test_split(X_adult, y_adult, test_size=0.2, random_state=42)

adult_model = LogisticRegression()
adult_model.fit(X_adult, y_adult)
#adult_model.fit(X_train_adult, y_train_adult)

# Train logistic regression model for county_df
X_county = pd.DataFrame(county_df)
y_county = county_df.pop('Income')

# Scale the input features of county_df using a standard scaler
#scaler_county = StandardScaler()
#X_county_scaled = scaler_county.fit_transform(X_county)
#X_train_county, X_test_county, y_train_county, y_test_county = train_test_split(X_county_scaled, y_county, test_size=0.2, random_state=42)

county_model = LogisticRegression()
county_model.fit(X_county, y_county)
#county_model.fit(X_train_county, y_train_county)

c:\Users\guill\anaconda3\envs\IREI\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [118]:
from sklearn.metrics import accuracy_score

y_pred_adult = adult_model.predict(X_adult)
print("Adults Dataset: ", accuracy_score(y_adult, y_pred_adult))

y_pred_county = county_model.predict(X_county)
print("County Dataset: ", accuracy_score(y_county, y_pred_county))

Adults Dataset:  0.7876907957372317
County Dataset:  0.7822981366459627


In [119]:
print(county_df['Income'])

KeyError: 'Income'

In [ ]:
adult_df.insert(loc=0, column='score', value=y_pred_adult)

adult_df = adult_df.rename(columns={"income": "label_value"})
cols = adult_df.columns.tolist()
cols.remove('label_value')
cols.insert(1, 'label_value')
adult_df = adult_df[cols]



county_df.insert(loc=0, column='score', value=y_pred_county)

county_df = county_df.rename(columns={"Income": "label_value"})

# cols = county_df.columns.tolist()
# cols.remove('label_value')
# cols.insert(1, 'label_value')
# county_df = county_df[cols]

# print(county_df)

# # Prepare data for Aequitas
# aequitas_county_df = X_test_county[['Men', 'Women', 'Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific']].copy()
# aequitas_county_df['label_value'] = y_test_county.values
# aequitas_county_df['score'] = y_pred_county

In [ ]:
# Visualize the distribution of income levels across gender and race for the adult_df model
fl.plot_distribution(adult_df, 'income', 'gender', 'race', model=adult_model, X_test=X_test_adult, y_test=y_test_adult)

# Visualize the distribution of income_per_capita_binary across gender and race for the county_df model
fl.plot_distribution(county_df, 'income_per_capita_binary', 'gender', 'race', model=us_census_model, X_test=X_test_us_census, y_test=y_test_us_census)



NameError: name 'fl' is not defined

In [ ]:
# Automatically generate a report
fscorer = fl.FairnessScorer(
    adult_df,
    target_attribute="income",
    sensitive_attributes=[
        "age",
        "race",
        "sex"
    ]
)
fscorer.demographic_report()

TypeError: __init__() got an unexpected keyword argument 'target_attribute'